In [ ]:
! apt-get install -y g++ openjdk-8-jdk python3-dev curl git build-essential
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [1]:
import torch
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

import ko_mrc.datasets as datasets
import ko_mrc.utils as utils
import ko_mrc.models as models

In [2]:
dataset = datasets.KoMRC.load('data/train.json')
print("Number of Questions", len(dataset))
print(dataset[0])

Number of Questions 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': [{'text': '한 달가량', 'answer_start': 478}, {'text': '한 달', 'answer_start': 478}]}


In [ ]:
## sentencepiece (subword tokenizer) add - Jinha

In [9]:
!pip install sentencepiece

     |████████████████████████████████| 1.1 MB 5.0 MB/s            


In [ ]:
import sentencepiece as spm

In [3]:
dataset = datasets.TokenizedKoMRC.load('data/train.json')
train_dataset, eval_dataset = datasets.TokenizedKoMRC.split(dataset)
print("Number of Questions", len(train_dataset), len(eval_dataset))
print(train_dataset[0])

Number of Questions 10834 1203
{'guid': '531b9879ffb54f64964b47fd832aa3af', 'context_original': '◇새로운 디지털 시대=세계적 정보기술(IT) 기업 구글의 에릭 슈밋 회장과 구글의 싱크탱크인 구글 아이디어의 제러드 코언 소장이 쓴 IT 시대 미래 예측서. 세계인이 온라인 세상에서 만나게 될 모습을 다뤘다. 지난해 초판 발행 이후 데이터 영구화, 중산층 일자리, 자유와 안보, 디지털 혁명 문제 등을 추가한 개정 증보판이다.(이진원 옮김, 알키, 520쪽, 2만원)◇종횡무진 역사=동양과 서양 문명을 비교하며 두 문명의 탄생, 만남, 융합을 그린 책. 과거 문명 탐구부터 문명의 발전 방향까지 살폈다. 책 이름에 맞게 일방통행적 역사 서술이 아니라 시·공간을 넘나들며 역사 현장을 누빈다. 역사를 현실과 함께 이해하려는 저자의 문제의식이 돋보인다.(남경태 지음, 휴머니스트, 744쪽, 3만8000원)◇영원의 철학=‘영원의 철학’은 16세기에 처음 언급된 ‘모든 위대한 종교의 본질적이고 공통된 핵심 진리’로, 세계의 종교적 전통이 공유하는 세계관, 인간관, 윤리관을 말한다.《멋진 신세계》의 올더스 헉슬리가 420개의 동서고금 문헌을 통해 영원의 철학을 소개한다. 인용문만 읽어도 흥미로운 종교서이자 명상서.(조옥경 옮김, 김영사, 528쪽, 1만9800원)◇대한민국 5천년 역사리더십을 말한다=고조선의 단군부터 이명박 전 대통령에 이르는 우리 민족의 리더십을 다룬 책. 리더십의 관점에서 우리 역사를 바라보며 역사 속 사건들에서 리더십이 어떤 힘을 발휘했는지 살펴보고 홍익인간, 민족주의, 민주주의 등 우리 민족 고유의 리더십 DNA 여덟 가지를 제시한다.(최익용 지음, 옥당, 436쪽, 2만2000원)◇한국 자본주의 모델=진보적 한국 경제론의 이론과 참여를 주도해 온 저자가 광복 이후 박근혜 정부까지 70여년의 대한민국 경제·자본주의 모델을 분석한 책. 이승만 정부의 대미 의존 경제, 박정희 개발주의의

In [4]:
sample = train_dataset[0]
print(sample['context'][sample['answers'][0]['start']:sample['answers'][0]['end']+1])

['420', '개']


In [8]:
tokenizer = utils.Tokenizer.build_vocab(dataset)
print(tokenizer.sample2ids(train_dataset[0]))

Counting Vocab: 100%|██████████| 12037/12037 [00:16<00:00, 717.17it/s]


KeyError: 'answers'

In [ ]:
train_dataset = utils.TokenizerWrapperDataset(train_dataset, tokenizer)
eval_dataset = utils.TokenizerWrapperDataset(eval_dataset, tokenizer)

print(train_dataset[0])

In [ ]:
collator = utils.Collator(tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collator, num_workers=4)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False, collate_fn=collator, num_workers=4)
batch = next(iter(train_loader))
print(batch['input_ids'].shape)
print(batch['input_ids'])
print(list(batch.keys()))

In [ ]:
from transformers import BertConfig

config = BertConfig(
     vocab_size=tokenizer.vocab_size,
     max_position_embeddings=1024,
     hidden_size=512,
     num_hidden_layers=6,
     num_attention_heads=8,
     intermediate_size=2048
)
model = models.BertForQuestionAnswering(config)
model.cuda()

In [ ]:
writer = SummaryWriter(log_dir='review')
os.makedirs('dump', exist_ok=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
step = 0

In [ ]:
for epoch in range(1, 31):
    print("Epoch", epoch)
    for batch in tqdm(train_loader, desc='Train'):
        del batch['guid'], batch['context'], batch['question'], batch['position']
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start')
        end = batch.pop('end')
        
        start_logits, end_logits = model(**batch)
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()

        step += 1
        writer.add_scalar('Train Loss', loss.item(), step)
        del batch, start, end, start_logits, end_logits, loss

    with torch.no_grad():
        losses = []
        for batch in tqdm(eval_loader, desc="Evaluation"):
            del batch['guid'], batch['context'], batch['question'], batch['position']
            batch = {key: value.cuda() for key, value in batch.items()}
            start = batch.pop('start')
            end = batch.pop('end')

            start_logits, end_logits = model(**batch)
            loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

            losses.append(loss.item())
            del batch, start, end, start_logits, end_logits, loss
        loss = sum(losses) / len(losses)
        writer.add_scalar('Eval Loss', loss, step)

    model.save_pretrained(f'dump/model.{epoch}')
        

In [ ]:
model = models.BertForQuestionAnswering.from_pretrained('dump/model.30')
model.cuda()
model.eval()

In [ ]:
for idx, raw_sample in zip(range(1, 4), train_dataset):
    sample = dict(raw_sample) 
    context = sample.pop('context')
    question = sample.pop('question')
    position = sample.pop('position')
    start, end = sample.pop('start'), sample.pop('end')
    del sample['guid']

    sample = {key: value.cuda()[None, :] for key, value in sample.items()}
    
    with torch.no_grad():
        start_logits, end_logits = model(**sample)
    start_logits.squeeze_(0), end_logits.squeeze_(0)

    print(f'------{idx}------')
    print('Context:', context)
    print('Question:', question)
    print('Answer:', tokenizer.logits2answer(raw_sample, start_logits, end_logits))

In [ ]:
test_dataset = datasets.TokenizedKoMRC.load('data/test.json')
test_dataset = utils.TokenizerWrapperDataset(test_dataset, tokenizer)
print("Number of Questions", len(test_dataset))
print(test_dataset[0])

In [ ]:
import os
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/baseline.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for raw_sample in tqdm(test_dataset, "Testing"):
        sample = dict(raw_sample) 
        guid = sample.pop('guid')
        context = sample.pop('context')
        position = sample.pop('position')
        start, end = sample.pop('start'), sample.pop('end')
        del sample['question']

        sample = {key: value.cuda()[None, :] for key, value in sample.items()}
        start_logits, end_logits = model(**sample)
        start_logits.squeeze_(0), end_logits.squeeze_(0)

        rows.append([guid, tokenizer.logits2answer(raw_sample, start_logits, end_logits)])
    
    writer.writerows(rows)